# Testes comparativos: GPU vs CPU

<p>O presente projeto busca realizar comparações de desempenho de processamento em CPU vs GPU para as tarefas de detecção facial e extração de embeddings faciais.</p>

<p>Nestes testes, que serão executados em python, utilizaremos a seguinte metodologia:</p>
<ol>
<li>Leitura de imagens em arquivos JPG, da ordem de 1000  registros;</li>
<li>Geração do DataFrame com os filenames e filepaths;</li>
<li>Medição do tempo de extração de embeddings de cada imagem obtida.</li>
</ol>

<p>Será utilizada biblioteca específica (timeit) para avaliação do desempenho do algoritmo relacionado tão somente ao tempo para a extração dos embeddings faciais.</p>


<p>Bibliotecas a serem utilizadas para processamento em CPU:</p>
<ul>
<li>timeit</li>
<li>functools.partial</li>
<li>Pandas</li>
<li>Numpy</li>
<li>Opencv</li>
<li>DeepFace com os modelos Dlib, Facenet512 etc.</li>
</ul>
<p>Obs: O processamento em CPU deverá ser realizado em ambiente sem as configurações de CUDA aplicadas.</p>
<hr>

<p>Bibliotecas a serem utilizadas para processamento em GPU:</p>
<ul>
<li>timeit</li>
<li>functools.partial</li>
<li>Pandas</li>
<li>Numpy</li>
<li>Opencv</li>
<li>DeepFace com os modelos Dlib, Facenet512 etc.</li>
</ul>
<p>Obs: O processamento em GPU deverá ser realizado em ambiente com as configurações de CUDA aplicadas.</p>

## Processamento em GPU/CPU



In [1]:
# Verifica se o CUDA esta instalado e a versão correspondente
!nvidia-smi

Tue Jun 25 22:45:16 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.03              Driver Version: 555.85         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 ...    On  |   00000000:01:00.0 Off |                  N/A |
| N/A   49C    P5             16W /   30W |       0MiB /   6144MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Import das bibliotecas
import pandas as pd
import numpy as np
import timeit
from functools import partial  # Para permitir a passagem de parametros dentro de timeit
import os
from PIL import Image
from deepface import DeepFace
import matplotlib.pyplot as plt
import tensorflow as tf;
# import cudf #nao utilizado, embora disponível.

print(tf.config.list_physical_devices('GPU'))
if tf.config.list_physical_devices('GPU'):
    print('GPU disponível.')

2024-06-25 22:45:19.242006: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-25 22:45:19.255893: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1719366319.274706 1150758 cuda_dnn.cc:8453] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1719366319.279745 1150758 cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-25 22:45:19.293248: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU disponível.


I0000 00:00:1719366322.399766 1150758 cuda_executor.cc:913] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1719366322.426508 1150758 cuda_executor.cc:913] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1719366322.426562 1150758 cuda_executor.cc:913] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:


PROCESSING_UNIT = 'CPU' # alterar para CPU ou GPU
IMAGES_PATH = './imagens/'      # Local das imagens
REPORT_PATH = './relatorios/'   # Local dos relatorios
N_LINES = 100                  # Qtd de imagens

# configurações básicas
if tf.config.list_physical_devices(PROCESSING_UNIT):
    print(PROCESSING_UNIT + ' disponível e configurada para uso.')

# Define os modelos
model_list = ['Dlib', 'Facenet512', 'Facenet', 'ArcFace', 'OpenFace', 'DeepID', 'DeepFace', 'GhostFaceNet', 'sFace'] # Face Embeddings Models
detector_list = ['opencv', 'ssd', 'dlib', 'mtcnn', 'retinaface','yolov8','yunet', 'fastmtcnn'] # Face Detection Models

CPU disponível e configurada para uso.


### Funções

In [4]:
def read_files_recursively(folder_path, data=[]):
    """Le arquivos JPG em uma pasta, recursivamente, e os armazena em uma lista.

    Args:
        folder_path (str): O path da pasta raiz onde procurar arquivos.
        data (list, optional):  Lista para armazenar as informações dos arquivos. Defaults = [].

    Returns:
        list: Uma list de dict, cada um contendo informações do arquivo.
    """

    for root, _, filenames in os.walk(folder_path):
        for filename in filenames:
            if filename.lower().endswith('.jpg'):  # Filter for .jpg files
                file_path = os.path.join(root, filename)
                try:
                    # Add any additional file processing or checks here if needed
                    data.append({
                        'filename': filename,
                        'relative_path': os.path.relpath(file_path, './')
                    })
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")  # Log errors 

    return data


# Função que extrai os embeddings faciais (vetores)
def gera_vetor(img_path, model_name='GhostFaceNet', detector_backend='yolov8', proc_unit='CPU'):
    with tf.device(f'/{PROCESSING_UNIT}:0'):
        # print(tf.device(f'/{PROCESSING_UNIT}:0'))
        return DeepFace.represent(img_path=img_path, model_name=model_name, detector_backend=detector_backend)



# Função que avalia o tempo de extração dos embeddings
def iteration_with_timing(df, row, model_name, detector_backend):  
    img_path = './' + row['relative_path'] 
    time_taken = timeit.timeit(partial(gera_vetor, img_path, model_name, detector_backend, proc_unit=PROCESSING_UNIT), number=1)
    df.loc[row.name, PROCESSING_UNIT+'_'+detector_backend+'_'+model_name] = time_taken  # Update DataFrame

### Leitura dos arquivos

In [5]:
# Lê os arquivos
file_data = read_files_recursively(IMAGES_PATH)

# Cria a DataFrame
df = pd.DataFrame(file_data).iloc[0:N_LINES]

# Mostra o DF
df

,filename,relative_path
0,001_fe3347c0.jpg,imagens/Celebrity Faces Dataset/Angelina Jolie...
1,002_8f8da10e.jpg,imagens/Celebrity Faces Dataset/Angelina Jolie...
2,003_57612506.jpg,imagens/Celebrity Faces Dataset/Angelina Jolie...
3,004_f61e7d0c.jpg,imagens/Celebrity Faces Dataset/Angelina Jolie...
4,005_582c121a.jpg,imagens/Celebrity Faces Dataset/Angelina Jolie...
...,...,...
95,096_75710434.jpg,imagens/Celebrity Faces Dataset/Angelina Jolie...
96,097_9a6bf61f.jpg,imagens/Celebrity Faces Dataset/Angelina Jolie...
97,098_dd1405fc.jpg,imagens/Celebrity Faces Dataset/Angelina Jolie...
98,099_c22d3e48.jpg,imagens/Celebrity Faces Dataset/Angelina Jolie...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   filename       100 non-null    object
 1   relative_path  100 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB


### Execução

In [7]:
# Inicia o loop
for d in detector_list:
    
    for m in model_list:
        
        print(f'running m:{m} | d: {d}...')
        print(tf.device(f'/{PROCESSING_UNIT}:0'))
        df[PROCESSING_UNIT+'_'+d+'_'+m] = None

        for index, row in df.iterrows():
            try:
                img_path = './' + row['relative_path'] 
                iteration_with_timing(df, row, m, d);  # Aciona a função de timing que irá acionar a função gera_vetor()

            except:
                print(f'linha {str(index)} nao tem face... m:{m} | d: {d}')

                pass

# Grava o resultado em um arquivo CSV            
df.to_csv(REPORT_PATH + PROCESSING_UNIT+'_all_models'+'_'+str(N_LINES)+'.csv', sep=';')

running m:Dlib | d: opencv...
linha 0 nao tem face... m:Dlib | d: opencv
linha 1 nao tem face... m:Dlib | d: opencv
linha 2 nao tem face... m:Dlib | d: opencv
linha 3 nao tem face... m:Dlib | d: opencv
linha 4 nao tem face... m:Dlib | d: opencv
linha 5 nao tem face... m:Dlib | d: opencv
linha 6 nao tem face... m:Dlib | d: opencv
linha 7 nao tem face... m:Dlib | d: opencv
linha 8 nao tem face... m:Dlib | d: opencv
linha 9 nao tem face... m:Dlib | d: opencv
linha 10 nao tem face... m:Dlib | d: opencv
linha 11 nao tem face... m:Dlib | d: opencv
linha 12 nao tem face... m:Dlib | d: opencv
linha 13 nao tem face... m:Dlib | d: opencv
linha 14 nao tem face... m:Dlib | d: opencv
linha 15 nao tem face... m:Dlib | d: opencv
linha 16 nao tem face... m:Dlib | d: opencv
linha 17 nao tem face... m:Dlib | d: opencv
linha 18 nao tem face... m:Dlib | d: opencv
linha 19 nao tem face... m:Dlib | d: opencv
linha 20 nao tem face... m:Dlib | d: opencv
linha 21 nao tem face... m:Dlib | d: opencv
linha 22 nao

I0000 00:00:1719366652.851678 1150758 cuda_executor.cc:913] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1719366652.851959 1150758 cuda_executor.cc:913] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1719366652.852024 1150758 cuda_executor.cc:913] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1719366654.984866 1150758 cuda_executor.cc:913] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1719366654.984933 1150758 cuda_executor.cc:913] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:171

linha 60 nao tem face... m:Facenet512 | d: opencv
linha 74 nao tem face... m:Facenet512 | d: opencv
running m:Facenet | d: opencv...
linha 60 nao tem face... m:Facenet | d: opencv
linha 74 nao tem face... m:Facenet | d: opencv
running m:ArcFace | d: opencv...
linha 60 nao tem face... m:ArcFace | d: opencv
linha 74 nao tem face... m:ArcFace | d: opencv
running m:OpenFace | d: opencv...
linha 60 nao tem face... m:OpenFace | d: opencv
linha 74 nao tem face... m:OpenFace | d: opencv
running m:DeepID | d: opencv...
linha 60 nao tem face... m:DeepID | d: opencv
linha 74 nao tem face... m:DeepID | d: opencv
running m:DeepFace | d: opencv...
linha 0 nao tem face... m:DeepFace | d: opencv
linha 1 nao tem face... m:DeepFace | d: opencv
linha 2 nao tem face... m:DeepFace | d: opencv
linha 3 nao tem face... m:DeepFace | d: opencv
linha 4 nao tem face... m:DeepFace | d: opencv
linha 5 nao tem face... m:DeepFace | d: opencv
linha 6 nao tem face... m:DeepFace | d: opencv
linha 7 nao tem face... m:Dee

### Verificação dos Resultados

In [8]:
# Exibe o DataFrame com os resultados
df.head()

,filename,relative_path,CPU_opencv_Dlib,CPU_opencv_Facenet512,CPU_opencv_Facenet,CPU_opencv_ArcFace,CPU_opencv_OpenFace,CPU_opencv_DeepID,CPU_opencv_DeepFace,CPU_opencv_GhostFaceNet,...,CPU_yunet_sFace,CPU_fastmtcnn_Dlib,CPU_fastmtcnn_Facenet512,CPU_fastmtcnn_Facenet,CPU_fastmtcnn_ArcFace,CPU_fastmtcnn_OpenFace,CPU_fastmtcnn_DeepID,CPU_fastmtcnn_DeepFace,CPU_fastmtcnn_GhostFaceNet,CPU_fastmtcnn_sFace
0,001_fe3347c0.jpg,imagens/Celebrity Faces Dataset/Angelina Jolie...,None,3.491603,2.198748,1.934824,2.060678,0.275469,None,4.458306,...,None,None,None,None,None,None,None,None,None,None
1,002_8f8da10e.jpg,imagens/Celebrity Faces Dataset/Angelina Jolie...,None,0.407442,0.338084,0.360428,0.241422,0.140599,None,0.42,...,None,None,None,None,None,None,None,None,None,None
2,003_57612506.jpg,imagens/Celebrity Faces Dataset/Angelina Jolie...,None,0.393832,0.338482,0.369146,0.213638,0.163696,None,0.384802,...,None,None,None,None,None,None,None,None,None,None
3,004_f61e7d0c.jpg,imagens/Celebrity Faces Dataset/Angelina Jolie...,None,0.28905,0.258803,0.267865,0.126679,0.061147,None,0.350876,...,None,None,None,None,None,None,None,None,None,None
4,005_582c121a.jpg,imagens/Celebrity Faces Dataset/Angelina Jolie...,None,0.373008,0.33229,0.326113,0.20209,0.136473,None,0.360807,...,None,None,None,None,None,None,None,None,None,None
